In [2]:
import face_recognition
import cv2
import os
import time
import threading
import queue
import socket
from datetime import datetime

In [3]:
known_faces_dir = 'train_images'
known_face_encodings = []
known_face_names = []

for person_name in os.listdir(known_faces_dir):
    person_dir = os.path.join(known_faces_dir, person_name)
    if os.path.isdir(person_dir):
        for image_name in os.listdir(person_dir):
            image_path = os.path.join(person_dir, image_name)
            # image = face_recognition.load_image_file(image_path)
            image = cv2.imread(image_path)
            image = cv2.resize(image, (0, 0), fx=0.25, fy=0.25)
            face_encodings = face_recognition.face_encodings(image)
            if not face_encodings:
                print("No face detected in", image_path)
                continue
            face_encoding = face_encodings[0]
            known_face_encodings.append(face_encoding)
            known_face_names.append(person_name)

In [4]:
known_face_names

['danny', 'keanu', 'pavan', 'vinay', 'vishal']

In [5]:
# from keras.models import load_model

# classifier = load_model('face recognisation.h5')


In [6]:

# import os
# import cv2
# import numpy as np
# from os import listdir
# from os.path import isfile, join


In [7]:
# image = cv2.imread('train_images/keanu/keanu.jpeg')
# # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# image = cv2.resize(image, (224,224))
# image = image / 255.
# image = image.reshape(1,224,224,3)

In [8]:
# start = time.time()
# pred = classifier.predict(image)
# end = time.time()
# print("Time taken in prediction: ", end - start)
# print(pred)

In [9]:
# known_face_names[np.argmax(pred)]

In [10]:
time.time()

1691495570.9497285

In [11]:
def recognize_faces(imager, q:queue.Queue, id:int, encounter_time:str):
    start = time.time()
    imager = cv2.resize(imager, (640, 480), fx=0.25, fy=0.25)
    face_locations = face_recognition.face_locations(imager, model="hog")
    face_encodings = face_recognition.face_encodings(imager, face_locations)
    for face_encoding, face_location in zip(face_encodings, face_locations):
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding, tolerance=0.6)        
        name = "Person"
        if True in matches:
            match_index = matches.index(True)
            name = known_face_names[match_index]
        else:
            return
        top, right, bottom, left = face_location
        cv2.rectangle(imager, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(imager, name, (left, top-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
    end = time.time()
    cv2.putText(imager, f"Time: {encounter_time}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    # cv2.imshow("Frame", imager)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # return imager
    q.put((id, encounter_time, imager))
    q.task_done()

In [12]:
import requests
import numpy as np
import imutils
import cv2

In [13]:
HOST = '192.168.0.106'
PORT = 8090

In [14]:
def send_frames(q:queue.Queue):
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind((HOST, PORT))
    server.listen(1)
    print('Server started')
    conn, addr = server.accept()
    print('Connected by', addr)
    while True:
        if not q.empty():
            frame = q.get()
            # q.task_done()
            frame_data = cv2.imencode('.jpg', frame)[1].tobytes()
            frame_size = len(frame_data)
            conn.sendall(frame_size.to_bytes(4, byteorder='big'))
            conn.sendall(frame_data)

In [15]:
def show_frames(q):
    while True:
        if not q.empty():
            frame = q.get()
            cv2.imshow('frame', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

In [16]:
url = "http://192.168.0.103:8080/shot.jpg"


In [17]:
width = 640
height = 480

In [19]:

def load_balancer(q:queue.Queue, queues:list[queue.Queue]):
    # i = 0
    while True:
        # if i==4:
            # return
        # i = 0
        for i in range(4):
            try:
                frame = q.get(block=False)
                threading.Thread(target=recognize_faces, args=(frame,queues[i], i, datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-3])).start()
            except:
                continue
                # i += 1
                

In [20]:
def show_frames_from_load_balancer(qs:list[queue.Queue]):
    while True:
        for q in qs:
            try:
                id, frame = q.get()
                cv2.imshow('frame', frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            except:
                pass
        # time.sleep(0.1)

In [21]:
def encounters(qs:list[queue.Queue], final_frames:queue.Queue):
    # i = 0
    while True:
        # if i==4:
            # break
        # i = 0
        for q in qs:
            try:
                id, encounter_time, frame = q.get(block=False)
                filename = 'encounters/' + str(id) + '/' + encounter_time +'.jpg'
                # cv2.imshow('Video', frame)
                # final_frames.put((id, encounter_time, frame))
                cv2.imwrite(filename, frame)
                # print(f'encounter {id} at {encounter_time} {type(frame)}')
                # while cv2.waitKey(1) & 0xFF != ord('q'):
                    # continue
            except:
                # i += 1
                continue

In [22]:
cam = cv2.VideoCapture(0)
queues = [queue.Queue() for _ in range(4)]
q = queue.Queue() 
final_frames = queue.Queue()
print("[INFO] starting video stream thread...")
threading.Thread(target=send_frames, args=(final_frames,)).start()
print("[INFO] starting load balancer thread...")
print("[INFO] starting recognition thread...")
threading.Thread(target=load_balancer, args=(q,queues)).start()
print("[INFO] starting encounters thread...")
threading.Thread(target=encounters, args=(queues,final_frames)).start()
end = 0
i=0
while i<120:
    img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    img = cv2.imdecode(img_arr, -1)
    img = imutils.resize(img, width=width, height=height)
    
    # ret, img = cam.read()
    # print(type(img))
    fps = 1 / (time.time() - end)   
    print('FPS:', fps)
    # img = cv2.resize(img, (height, width), fx=0.25, fy=0.25)
    end = time.time()
    q.put(img)
    final_frames.put(img)
    cv2.imshow('Camera', img)
    cv2.imwrite('test_images/'+str(i)+'.jpg', img)
    # cv2.imwrite('encounters/test'+str(i)+'.jpg', img)
    # threading.Thread(target=recognize_faces, args=(img,q)).start()
    # cv2.imshow('Camera', img)
    if cv2.waitKey(1) == ord('q'):
        break
    i+=1
  
cv2.destroyAllWindows()

[INFO] starting video stream thread...
[INFO] starting load balancer thread...
[INFO] starting recognition thread...
Server started
[INFO] starting encounters thread...
FPS: 5.91192784195273e-10
FPS: 1.6395239705344875
Connected by ('192.168.0.100', 37082)
FPS: 1.0028992063706503
FPS: 0.8774184262923621
FPS: 0.9452146511624239
FPS: 1.6070481232731904
FPS: 1.089949105612675
FPS: 1.4221807496661814
FPS: 1.1054523526321907
FPS: 0.7768166125212502
FPS: 0.6659069252158484
FPS: 0.860974411166156
FPS: 0.750521157472405
FPS: 0.5096191231589916
FPS: 1.0195704939113397
FPS: 0.6400606502782933
FPS: 1.2355302615222374
FPS: 1.6007022123505414
FPS: 1.4557863835181106
FPS: 1.5239363103285526
FPS: 1.6125978051958487
FPS: 1.1234256373719531
FPS: 0.9010958886451671
FPS: 1.089010129344982
FPS: 0.9200256465431675
FPS: 1.3876807758273868
FPS: 1.3061057731981236
FPS: 1.2071054621160688
FPS: 1.5025931645922426
FPS: 0.9440230834501836
FPS: 1.01324515470074
FPS: 1.1812744672821391
FPS: 0.8277401116006453
FPS: 

Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Users\saivishal radham\AppData\Local\Programs\Python\Python39\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\saivishal radham\AppData\Local\Programs\Python\Python39\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\saivishal radham\AppData\Local\Temp\ipykernel_8084\1673903670.py", line 14, in send_frames
ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine


FPS: 0.539960210108342
FPS: 1.050963526391079
FPS: 1.6844032909746383
FPS: 1.143307298672971
FPS: 1.5975501435938846
FPS: 0.9670596416282314
FPS: 1.2038891354153691
FPS: 1.2564680648379203
FPS: 1.0845959746149498
FPS: 1.7732313890785487
FPS: 1.4266655555309897
FPS: 1.4994324805586607
FPS: 2.4833810849232276
FPS: 1.6571818255810187
FPS: 1.0991100579100446
FPS: 1.773131688394481
FPS: 1.6763436455233798
FPS: 1.4579304241491418
FPS: 1.9494871231160158
FPS: 1.305294055771913
FPS: 1.8308318724988302
FPS: 2.129852693326996
FPS: 2.048563154812578
FPS: 1.4158924944907747


In [ ]:
def capture_video(camera):
    

In [257]:
load_balancer(q, queues)

In [261]:
encounters(queues)

encounter 0 at 20230808_154034_662 <class 'numpy.ndarray'>
encounter 1 at 20230808_154034_738 <class 'numpy.ndarray'>
encounter 2 at 20230808_154034_608 <class 'numpy.ndarray'>
encounter 3 at 20230808_154034_571 <class 'numpy.ndarray'>
encounter 0 at 20230808_154034_565 <class 'numpy.ndarray'>
encounter 1 at 20230808_154034_566 <class 'numpy.ndarray'>
encounter 2 at 20230808_154034_567 <class 'numpy.ndarray'>
encounter 3 at 20230808_154034_630 <class 'numpy.ndarray'>
encounter 0 at 20230808_154034_573 <class 'numpy.ndarray'>
encounter 1 at 20230808_154034_575 <class 'numpy.ndarray'>


In [258]:
q.qsize()

0

In [260]:
for i in range(4):
    print(queues[i].qsize())

3
3
2
2


In [ ]:
while True:
    frame = q.get()
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()


In [ ]:
threading.Thread(target=send_frames, args=(q,)).start()
cam = cv2.VideoCapture(0)
while True:
    # img_resp = requests.get(url)
    # img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    # img = cv2.imdecode(img_arr, -1)      
    ret, img = cam.read()
    img = imutils.resize(img, width=width, height=height)
    # img = cv2.resize(img, (width, height), fx=0.5, fy=0.5)
    threading.Thread(target=recognize_faces, args=(img,q)).start()
    cv2.imshow('Camera', img)
    if cv2.waitKey(1) == ord('q'):
        break
  
cv2.destroyAllWindows()

In [ ]:
import base64
import cv2
from io import BytesIO
from PIL import Image
import numpy as np
import websocket

def on_message(ws, message):
    frame_base64 = message
    image_data = base64.b64decode(frame_base64)
    img = Image.open(BytesIO(image_data))
    cv2.imshow('frame', cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        ws.close()

if __name__ == "__main__":
    ws_url = 'ws://192.168.0.100:8000/ws/video'
    ws = websocket.WebSocketApp(ws_url, on_message=on_message)
    ws.run_forever()